# Using OpenVINO Docker containers for development and deployment


## Introduction

The purpose of this tutorial is to examine a sample application that was created using the [Intel® Distribution of OpenVINO™ toolkit](https://software.intel.com/openvino-toolkit).  

This tutorial will go step-by-step to demonstrate [Speech Recognition DeepSpeech demo](https://docs.openvino.ai/latest/omz_demos_speech_recognition_deepspeech_demo_python.html) using OpenVINO™ Docker container. Recognition is performed using a pre-trained network - Mozilla* DeepSpeech. We will also cover model download and conversion via Model Optimizer. The model is capable of recognizing the speech in English.  Inference will be executed using the same CPU(s) running this Jupyter* Notebook.


## Prerequisites

This tutorial requires the following:
- Unix* host
- Installed and run [Docker](https://docs.docker.com/engine/install/)* engine/service on the host


## Preparation

OpenVINO has a couple of dependencies which need to be present on your computer. Additionally, to install some of them, you need to have root/admin rights. This might not be desirable. Using Docker represents much cleaner way. Especially when there is an image prepared for you on [Docker Hub OpenVINO organization](https://hub.docker.com/r/openvino).

We have some types of Docker images depend on distribution:

*   **runtime** - contains OpenVINO Runtime (Inference Engine core, nGraph) libs for each supported device (CPU, GNA, GPU, VPU)
*   **dev** - contains runtime part plus samples and Python development tools: Model Optimizer, Post training Optimization tool (POT), Accuracy checker, Open Model Zoo tools (downloader, converter)

You can search OpenVINO Docker images on Docker Hub or via cli:





```
docker search --format "{{.Name}}: {{.Description}}" --no-trunc --limit 100  openvino | grep ^openvino/
```




Output will be like this:
```
openvino/workbench: OpenVINO™ DL Workbench is a web GUI to simplify DL models inference and tuning on Intel® devices.
openvino/ubuntu18_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu20_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/model_server: Intel® Distribution of OpenVINO™ Model Server Docker images
openvino/cvat_ui: Computer Vision Annotation Tool (CVAT) frontend UI image
openvino/ubuntu18_data_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/cvat_server: Computer Vision Annotation Tool (CVAT) annotation backend image
openvino/ubuntu18_dev_no_samples: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_data_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 18.04 LTS
openvino/ubuntu18_model_server: [DEPRECATED] Please use openvino/model_server instead
openvino/ubuntu20_data_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/ubuntu20_data_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/winserver2019_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for Windows Server Core base LTSC 2019
openvino/ubuntu20_dev: Intel® Distribution of OpenVINO™ toolkit Docker image for Ubuntu* 20.04 LTS
openvino/ubuntu18_base_cpu: [DEPRECATED] Not maintained anymore
openvino/centos7_runtime: Intel® Distribution of OpenVINO™ toolkit Docker image for CentOS*
```



## Part 1. Using OpenVINO development Docker container for model conversion and execution.

### 1.	Pull dev OpenVINO docker image and run it to download a model.

**Note:** it will write a model on the host, please use /tmp folder.

`docker run` command will pull the 2022.1.0 release of openvino/ubuntu20_dev image from Docker Hub and run a container with `omz_downloader` script to download `mozilla-deepspeech-0.6.1` model to `/tmp/model` folder on the host.

* `-u root` or `-u 0` option needs to save downloaded model in `/tmp/model` folder under root user.
* `--rm` option needs to remove container after execution.
* `-v /tmp/model:/tmp/model` option needs to mount host folder to the container.


In [10]:
!mkdir /tmp/model
!docker run -u root --rm -v /tmp/model:/tmp/model openvino/ubuntu20_dev:2022.1.0 omz_downloader --name mozilla-deepspeech-0.6.1 -o /tmp/model

################|| Downloading mozilla-deepspeech-0.6.1 ||################

========== Downloading /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models.tar.gz
... 0%, 1024 KB, 1347 KB/s, 0 seconds passed
... 0%, 2048 KB, 1617 KB/s, 1 seconds passed
... 0%, 3072 KB, 1723 KB/s, 1 seconds passed
... 0%, 4096 KB, 1786 KB/s, 2 seconds passed
... 0%, 5120 KB, 1826 KB/s, 2 seconds passed
... 0%, 6144 KB, 1856 KB/s, 3 seconds passed
... 0%, 7168 KB, 1876 KB/s, 3 seconds passed
... 0%, 8192 KB, 1893 KB/s, 4 seconds passed
... 0%, 9216 KB, 1902 KB/s, 4 seconds passed
... 0%, 10240 KB, 1913 KB/s, 5 seconds passed
... 0%, 11264 KB, 1922 KB/s, 5 seconds passed
... 1%, 12288 KB, 1932 KB/s, 6 seconds passed
... 1%, 13312 KB, 1933 KB/s, 6 seconds passed
... 1%, 14336 KB, 1932 KB/s, 7 seconds passed
... 1%, 15360 KB, 1939 KB/s, 7 seconds passed
... 1%, 16384 KB, 1944 KB/s, 8 seconds passed
... 1%, 17408 KB, 1946 KB/s, 8 seconds passed
... 1%, 18432 KB, 1947 KB/s, 9 seconds passed
... 1%, 

### 2. Convert our model to the Intermediate Representation (IR)

OpenVINO™ toolkit introduces its own format of graph representation and its own operation set. A graph is represented with two files: an XML file and a binary file. This representation is commonly referred to as the Intermediate Representation or IR.

`docker run` command will run a container with `omz_converter` to convert the model stored in `/tmp/model` folder and save it to `/tmp/model/converted` folder.

In [11]:
!docker run -u 0 --rm -v /tmp/model:/tmp/model openvino/ubuntu20_dev:2022.1.0 omz_converter --name mozilla-deepspeech-0.6.1 -d /tmp/model -o /tmp/model/converted

========== Converting mozilla-deepspeech-0.6.1 to IR (FP16)
Conversion command: /usr/bin/python3.8 -- /usr/local/bin/mo --framework=tf --data_type=FP16 --output_dir=/tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16 --model_name=mozilla-deepspeech-0.6.1 --input=input_node,previous_state_h,previous_state_c --input_model=/tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/output_graph.pb '--freeze_placeholder_with_value=input_lengths->[16]' --output=logits,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd,cudnn_lstm/rnn/multi_rnn_cell/cell_0/cudnn_compatible_lstm_cell/GatherNd_1 --disable_nhwc_to_nchw '--layout=input_node(NSTC),previous_state_h(NC),previous_state_c(NC)' '--input_shape=[1, 16, 19, 26],[1, 2048],[1, 2048]'

[ WARNING ]  Use of deprecated cli option --disable_nhwc_to_nchw detected. Option use in the following releases will be fatal. 
Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/tmp/model/public/mozi

### 3. Execute the Speech Recognition Demo

We need execute 5 steps to run speech demo in isolated environment:

1. Download Open Model Zoo demos from tag or branch corresponding with OpenVINO version you use.
2. Download the example audio sample [how_are_you_doing.wav](https://raw.githubusercontent.com/dlstreamer/dlstreamer/v1.5.3/samples/gst_launch/audio_detect/how_are_you_doing.wav)
3. Install the model API package and third-party Python dependencies for the demo.
4. Build ctcdecode_numpy Python module, which implements CTC decoding in C++ for faster decoding.
5. Run Speech Recognition Demo with converted model, the audio sample and pre/post-processing profile `-p mds06x_en` for Mozilla DeepSpeech v0.6.x model.

`docker run` command will run a container with `/bin/bash` session to execute 3 mentioned steps above.

In [12]:
!git clone --depth=1 --recurse-submodules --shallow-submodules https://github.com/openvinotoolkit/open_model_zoo.git /tmp/model/open_model_zoo
!wget -q --show-progress -P /tmp/model https://raw.githubusercontent.com/dlstreamer/dlstreamer/v1.5.3/samples/gst_launch/audio_detect/how_are_you_doing.wav
!docker run -u 0 --rm -v /tmp/model:/tmp/model openvino/ubuntu20_dev:2022.1.0 /bin/bash -c "python3 -m pip install --user /tmp/model/open_model_zoo/demos/common/python && python3 -m pip install --user -r /tmp/model/open_model_zoo/demos/requirements.txt && cd /tmp/model/open_model_zoo/demos/speech_recognition_deepspeech_demo/python/ctcdecode-numpy && python3 -m pip install . && cd ../ && python3 speech_recognition_deepspeech_demo.py -m /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/mozilla-deepspeech-0.6.1.xml -L /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/lm.binary -p mds06x_en -i /tmp/model/how_are_you_doing.wav --realtime"

Cloning into '/tmp/model/open_model_zoo'...
remote: Enumerating objects: 3144, done.
remote: Counting objects: 100% (3144/3144), done.
remote: Compressing objects: 100% (2344/2344), done.
remote: Total 3144 (delta 971), reused 1447 (delta 681), pack-reused 0
Receiving objects: 100% (3144/3144), 107.52 MiB | 1.94 MiB/s, done.
Resolving deltas: 100% (971/971), done.
Submodule 'demos/thirdparty/gflags' (https://github.com/gflags/gflags.git) registered for path 'demos/thirdparty/gflags'
Cloning into '/tmp/model/open_model_zoo/demos/thirdparty/gflags'...
remote: Enumerating objects: 67, done.        
remote: Counting objects: 100% (67/67), done.        
remote: Compressing objects: 100% (61/61), done.        
remote: Total 67 (delta 3), reused 29 (delta 0), pack-reused 0        
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38), done.
remote: Compressing objects: 100% (19/19), done.
remote: Total 20 (de

## Part 2. Set up OpenVINO runtime container to execute our demo

Let's imagine that we need to deploy this speech recognizer to production. OpenVINO provides a lightweight runtime container for this purpose. 



First, we need download the demo application and audio sample to the host `/tmp/demo` folder:

OpenVINO runtime image is 4 times lighter than dev image:

```
Image                               Hash           Size

openvino/ubuntu20_dev:2022.1.0      3b16f830655f   5.49GB

openvino/ubuntu20_runtime:2022.1.0  8e320c9d4e99   1.48GB
```

In [14]:
!mkdir /tmp/demo
!wget -q --show-progress -P /tmp/demo https://raw.githubusercontent.com/dlstreamer/dlstreamer/v1.5.3/samples/gst_launch/audio_detect/how_are_you_doing.wav
!git clone --depth=1 --recurse-submodules --shallow-submodules https://github.com/openvinotoolkit/open_model_zoo.git /tmp/demo/open_model_zoo

how_are_you_doing.w 100%[===================>] 106.45K  --.-KB/s    in 0.07s   
Cloning into '/tmp/demo/open_model_zoo'...
remote: Enumerating objects: 3144, done.
remote: Counting objects: 100% (3144/3144), done.
remote: Compressing objects: 100% (2323/2323), done.
remote: Total 3144 (delta 971), reused 1469 (delta 702), pack-reused 0
Receiving objects: 100% (3144/3144), 107.53 MiB | 1.90 MiB/s, done.
Resolving deltas: 100% (971/971), done.
Submodule 'demos/thirdparty/gflags' (https://github.com/gflags/gflags.git) registered for path 'demos/thirdparty/gflags'
Cloning into '/tmp/demo/open_model_zoo/demos/thirdparty/gflags'...
remote: Enumerating objects: 67, done.        
remote: Counting objects: 100% (67/67), done.        
remote: Compressing objects: 100% (61/61), done.        
remote: Total 67 (delta 3), reused 29 (delta 0), pack-reused 0        
remote: Total 0 (delta 0), reused 0 (delta 0), pack-reused 0
remote: Enumerating objects: 38, done.
remote: Counting objects: 100% (38/38

Second, let's try to run it on runtime OpenVINO image. We need repeat 5 steps mentioned above for dev image and install compiler, Python development tools to build pip wheel for ctcdecode-numpy and install `libsndfile1` library for reading/writing audio files:

In [15]:
!docker run -u 0 --rm -v /tmp/demo:/tmp/demo -v /tmp/model:/tmp/model  openvino/ubuntu20_runtime:2022.1.0 /bin/bash -c "apt update && apt install -y gcc g++ python3-dev && python3 -m pip install --user /tmp/model/open_model_zoo/demos/common/python && python3 -m pip install --user -r /tmp/model/open_model_zoo/demos/requirements.txt && cd /tmp/demo/open_model_zoo/demos/speech_recognition_deepspeech_demo/python/ctcdecode-numpy && python3 -m pip install . && cd ../ && python3 speech_recognition_deepspeech_demo.py -m /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/mozilla-deepspeech-0.6.1.xml -L /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/lm.binary -p mds06x_en -i /tmp/demo/how_are_you_doing.wav --realtime"



Get:1 http://archive.ubuntu.com/ubuntu focal InRelease [265 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:4 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Get:5 http://archive.ubuntu.com/ubuntu focal/main amd64 Packages [1275 kB]
Get:6 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [1686 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal/universe amd64 Packages [11.3 MB]
Get:8 http://security.ubuntu.com/ubuntu focal-security/multiverse amd64 Packages [25.8 kB]
Get:9 http://security.ubuntu.com/ubuntu focal-security/universe amd64 Packages [863 kB]
Get:10 http://security.ubuntu.com/ubuntu focal-security/restricted amd64 Packages [1069 kB]
Get:11 http://archive.ubuntu.com/ubuntu focal/multiverse amd64 Packages [177 kB]
Get:12 http://archive.ubuntu.com/ubuntu focal/restricted amd64 Packages [33.4 kB]
Get:13 http://archive.ubuntu.com/ubuntu f

Third, let's save our lightweight runtime image. In that case, we should execute some additional steps:


1.   Run a container in interactive mode and set a name for it to use it later:


```
docker run -u 0 -it --name speech --rm -v /tmp/demo:/tmp/demo -v /tmp/model:/tmp/model openvino/ubuntu20_runtime:2022.1.0
```




2.   Setup environment for the speech sample.


```
apt update && apt install -y gcc g++ python3-dev && python3 -m pip install --user /tmp/model/open_model_zoo/demos/common/python && python3 -m pip install --user -r /tmp/model/open_model_zoo/demos/requirements.txt && cd /tmp/demo/open_model_zoo/demos/speech_recognition_deepspeech_demo/python/ctcdecode-numpy && python3 -m pip install . && cd -
```



3.   Copy demo and model in the image



```
cp -rf /tmp/demo/open_model_zoo/demos/speech_recognition_deepspeech_demo/python/* /opt/intel/openvino && cp -rf /tmp/demo/how_are_you_doing.wav /opt/intel/openvino && cp -rf /tmp/model/converted/public/mozilla-deepspeech-0.6.1/FP16/* /opt/intel/openvino && cp -rf /tmp/model/public/mozilla-deepspeech-0.6.1/deepspeech-0.6.1-models/* /opt/intel/openvino
```



4.   Remove build artifacts



```
apt autoremove -y gcc g++ && rm -rf /var/lib/apt/lists/*
```



Now, our container is ready to be saved as a new image. Up another console and run `docker commit`:

```sh
docker commit speech ubuntu20_runtime:speech
```




Created image (Mozilla* DeepSpeech is a very big, plus 2 Gb to the runtime image):
```
Name                                 Hash                Size
ubuntu20_runtime:speech              6451dd798120        4GB
```


We can run the demo as non-root user with our speech image: 


```
docker run --rm ubuntu20_runtime:speech /bin/bash -c "python3 speech_recognition_deepspeech_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav --realtime"
```



Of course, you can run a container and inference on different Intel devices like iGPU (Intel® Processor Graphics), NCS2(Intel® Neural Compute Stick 2) or HDDL (Intel® Vision Accelerator Design with Intel® Movidius™ VPUs). 

To enable GPU access, add `--device /dev/dri` to `docker run` command and `-d GPU` to demo command line. GPU will be shared as device with container:


```
docker run --rm --device /dev/dri ubuntu20_runtime:speech /bin/bash -c "python3 speech_recognition_deepspeech_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav --realtime -d GPU"
```


To run inference on the NCS2 USB stick, add `--device-cgroup-rule='c 189:* rmw' -v /dev/bus/usb:/dev/bus/usb` to `docker run` command, and `-d MYRIAD` to demo command line. Mount USB stick will be shared as volume and set up the rule to the cgroup allowed 189 USB char devices the following access rights (r - read, m – mknod – create a device and w - write):

```
docker run --rm --device-cgroup-rule='c 189:* rmw' -v /dev/bus/usb:/dev/bus/usb ubuntu20_runtime:speech /bin/bash -c "python3 speech_recognition_deepspeech_demo.py -m mozilla-deepspeech-0.6.1.xml -L lm.binary -p mds06x_en -i how_are_you_doing.wav --realtime -d MYRIAD"
```

You can find more information about how to run a container with access to a specific device on our [Docker CI GitHub repo](https://github.com/openvinotoolkit/docker_ci/blob/master/get-started.md#run-a-container)

## Summary

In this article, we briefly introduced the speech recognition demo using OpenVINO Docker images. Of course, there is much more to try. We hope that this article has motivated you to try it yourself and maybe continue to explore all the possibilities of OpenVINO Docker images.



## References



*   [Available OpenVINO Docker images](https://github.com/openvinotoolkit/docker_ci#prebuilt-images)
*   [Docker CI framework for Intel® Distribution of OpenVINO™ toolkit](https://github.com/openvinotoolkit/docker_ci). The Framework can generate a Dockerfile, build, test, and deploy an image with the Intel® Distribution of OpenVINO™ toolkit. You can reuse available Dockerfiles, add your layer and customize the image of OpenVINO™ for your needs.

